## This Notebook will pull in and summarize our Pit Robot Scouter JSON Data.

The Notebook exports its results to CSV and Excel files located in a 'LatestResults' directory 
relative to this Jupyter Notebook.

** NO NEED TO RUN THE NEXT CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES **

In [1]:
# NO NEED TO RUN THIS CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES. 
# No problem though if you accidently run it as it will just say the "Requirement already satisfied"

!pip install requests
!pip install pandas

# The following library is currently being used for the Excel writer.  Was planning to use for exporting 
# to an Excel file format. Panda no longer supports exporting directly into an Excel file format.
!pip install openpyxl

In [2]:
import requests
import pandas as pd
from datetime import datetime

In [3]:
# Getting a unique file timestamp
currentTimeStamp = datetime.now()
file_timestamp = currentTimeStamp.strftime("%Y-%m-%d-T%H-%M")

In [4]:
df_scouting = pd.read_json('LatestInputOfScoutData/PIT.json')
display(df_scouting)

,teams
1410,"[{'name': 'Pit 1', 'timestamp': 1647646748582,..."
1619,"[{'name': 'Pit 1', 'timestamp': 1647470571192,..."
2945,"[{'name': 'Pit 1', 'timestamp': 1647742305729,..."
3374,"[{'name': 'Pit 1', 'timestamp': 1647741329171,..."
4418,"[{'name': 'Pit 1', 'timestamp': 1647796733933,..."
4550,"[{'name': 'Pit 1', 'timestamp': 1647793791192,..."
5690,"[{'name': 'Pit 1', 'timestamp': 1647743278733,..."


In [5]:
df_scouting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 1410 to 5690
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   teams   7 non-null      object
dtypes: object(1)
memory usage: 112.0+ bytes


In [6]:
# Making it easier to designate the header / column name to use per metric collected.
d_labels = {'ECIgrKJfMO8I2E8Lsovh': 'Scout', 
            'ONV3DyLTXyEeLzuZZnQs': 'HARDWARE Floor Intake', 'TvyFAXJuHxPsW26qxSXm': 'HARDWARE Drivetrain', 
            'ImxowXvvNc0k4foZnkC2': 'AUTO Level', 'RdwMEdp9hqmT2nF50pn8': 'AUTO Strategy Comments',
            'zQCQI1X0XFbAdupCuZhm': 'TELEOP Where They Shoot', 'ZKt8i4uYZsEAus75zJ7o': 'TELEOP Strategy Comments',
            'tlJnSqtvEgakktTXBgkW': 'END GAME Highest Climb', 'DNmVOXCuMUKHvU6BzLMV': 'END GAME Climb Comments', 
            'LY1pu680oPokOpJNI4q3': 'DEFENSE Shown', 'MJIMd2Nj0Cf9zPEzsRYh': 'DEFENSE Comments',
            'ry5Q83c1ty8QyWcoQKsI': 'Additional Notes'
           }       

In [7]:
# Index to use for each row of pit strategy that scouts have provided
idx_allpitstrategies = 0

# A pandas DataFrame to collect all the metrics.  
# Each row is a match and we will identify the team that was scouted in the row. 
df_allpitstrategies = pd.DataFrame()

# We will collect all the metrics for the current match into a dictionary.
# This dictionary will be added to the end of the df_allmetrics dataframe.
dict_pitstrategy = {}
    
# Let's run through all the scouting match metrics data we have from our scouts
for index_team, row_team in df_scouting.iterrows():
    
    # Let's grab all the matches of metrics that we have for the current team
    df_matches = pd.DataFrame.from_dict(row_team['teams'])
    display(df_matches)
    
    # We'll now iterate through each match's metrics for the current team
    for index_match, row_match in df_matches.iterrows():
        
        # Setting the team number and timestamp for the current match's metrics
        team = index_team
        timestamp = pd.Timestamp(row_match['timestamp'], unit='ms')  
        
        # Adding the team number and timestamp into the dictionary that we will add as a row of metrics
        dict_pitstrategy['Index'] = [idx_allpitstrategies]        
        dict_pitstrategy['Team'] = [team]
        dict_pitstrategy['Timestamp'] = [timestamp.strftime('%m-%d-%Y')]
        
        # When the JSON file was imported, the metrics were still in a set of key, value, category, name set of fields
        # under the 'metrics' column of the dataframe that was created when we read the JSON file.
        # We will iterate through these key, value type sets of data to parse out what we want from the scouting metrics.
        d_metrics = row_match['metrics']
        
        # Using items() + list comprehension
        # Substring Key match in dictionary
        for key, val in d_metrics.items():
            # print("key: ", key, "Type:", val['type'],"Name:", val['name'], "Category:", val['category'],"Value:", val['value'])
            # print("Label will be:", d_labels[key], 'Index:', i)

            # Adding metrics to our dictionary that will be added as a row once we have all of them for the match.
            if val['value'] is None:
                dict_pitstrategy[d_labels[key]] = ['']
            elif val['type'] in {"boolean", "number","stopwatch"}:
                dict_pitstrategy[d_labels[key]] = [val['value']]                
            else:
                dict_pitstrategy[d_labels[key]] = [val['value']]                

        print("Dict: ", dict_pitstrategy)
        
        # Let's convert our dictionary of metrics for a match into a DataFrame to be ready to add
        new_pitstrategy = pd.DataFrame(dict_pitstrategy)
        
        # If this is our first row, let's initialize our all metrics DataFrame to this first row.
        if df_allpitstrategies.empty:
            # If first row let's set it
            df_allpitstrategies = new_pitstrategy
        else:
            #append new row to the dataframe
            df_allpitstrategies = pd.concat([df_allpitstrategies, new_pitstrategy], ignore_index=True)
    
        # Incrementing to be ready to use as an index to each row
        idx_allpitstrategies = idx_allpitstrategies + 1

# Let's just see what we have!
display(df_allpitstrategies)

,name,timestamp,metrics
0,Pit 1,1647646748582,"{'ECIgrKJfMO8I2E8Lsovh': {'type': 'text', 'nam..."


Dict:  {'Index': [0], 'Team': [1410], 'Timestamp': ['03-18-2022'], 'Scout': [''], 'HARDWARE Floor Intake': [True], 'HARDWARE Drivetrain': ['Unknown '], 'AUTO Level': ['Unknown '], 'AUTO Strategy Comments': [''], 'TELEOP Where They Shoot': ['Unknown '], 'TELEOP Strategy Comments': [''], 'END GAME Highest Climb': ['Unknown '], 'END GAME Climb Comments': [''], 'DEFENSE Shown': [False], 'DEFENSE Comments': [''], 'Additional Notes': ['']}


,name,timestamp,metrics
0,Pit 1,1647470571192,"{'ECIgrKJfMO8I2E8Lsovh': {'type': 'text', 'nam..."


Dict:  {'Index': [1], 'Team': [1619], 'Timestamp': ['03-16-2022'], 'Scout': ['JB'], 'HARDWARE Floor Intake': [True], 'HARDWARE Drivetrain': ['Swerve '], 'AUTO Level': ['5-More than 2 Cargo combination'], 'AUTO Strategy Comments': ["Setup on the opposite side as the hangar in their Alliance's tarmac."], 'TELEOP Where They Shoot': ['Upper Hub only'], 'TELEOP Strategy Comments': ['They like the launchpad when heavily defended.  Otherwise typically just outside the tarmac.  They go to the opposite side of the field to gather cargo.'], 'END GAME Highest Climb': ['Traversal '], 'END GAME Climb Comments': ['15 sec climb. They go into the Hangar with 30 seconds remaining.'], 'DEFENSE Shown': [False], 'DEFENSE Comments': ['Never seen them play defense.'], 'Additional Notes': ['']}


,name,timestamp,metrics
0,Pit 1,1647742305729,"{'ECIgrKJfMO8I2E8Lsovh': {'type': 'text', 'nam..."


...

,name,timestamp,metrics
0,Pit 1,1647793791192,"{'ECIgrKJfMO8I2E8Lsovh': {'type': 'text', 'nam..."


Dict:  {'Index': [5], 'Team': [4550], 'Timestamp': ['03-20-2022'], 'Scout': ['NG'], 'HARDWARE Floor Intake': [False], 'HARDWARE Drivetrain': ['Unknown '], 'AUTO Level': ['3-Shoot + Taxi only (1 Cargo)'], 'AUTO Strategy Comments': ['Changed their auto routine to shoot into the low hub partway through quals.'], 'TELEOP Where They Shoot': ['Nowhere'], 'TELEOP Strategy Comments': ['Did not pick up or shoot at all during teleop in AZ.'], 'END GAME Highest Climb': ['Traversal '], 'END GAME Climb Comments': ['Enter hangar at 30sec. Have to enter the hangar from the side, they are too tall to go under the low bar.'], 'DEFENSE Shown': [True], 'DEFENSE Comments': ['Good at blocking. Never got penalties (except for that one time :). Not as mobile, so they have trouble blocking swerve robots. '], 'Additional Notes': ['In the first couple qual matches their robot stopped working. Very bouncy and tippy when stopping.']}


,name,timestamp,metrics
0,Pit 1,1647743278733,"{'ECIgrKJfMO8I2E8Lsovh': {'type': 'text', 'nam..."


Dict:  {'Index': [6], 'Team': [5690], 'Timestamp': ['03-20-2022'], 'Scout': ['NG'], 'HARDWARE Floor Intake': [True], 'HARDWARE Drivetrain': ['Unknown '], 'AUTO Level': ['4-Shoot + Taxi + Intake (2 Cargo max)'], 'AUTO Strategy Comments': ['Looked like they were working on picking up a third ball from the terminal and shooting it during auto. '], 'TELEOP Where They Shoot': ['Upper Hub only'], 'TELEOP Strategy Comments': [''], 'END GAME Highest Climb': ['Mid'], 'END GAME Climb Comments': [''], 'DEFENSE Shown': [True], 'DEFENSE Comments': ['Did well in finals when playing defense '], 'Additional Notes': ["It takes them awhile to line up to shoot. They always shoot from just outside the tarmac tape. It looks like they have trouble moving around when they are near the power cord on the floor. They don't do well when they have defense against them. "]}


,Index,Team,Timestamp,Scout,HARDWARE Floor Intake,HARDWARE Drivetrain,AUTO Level,AUTO Strategy Comments,TELEOP Where They Shoot,TELEOP Strategy Comments,END GAME Highest Climb,END GAME Climb Comments,DEFENSE Shown,DEFENSE Comments,Additional Notes
0,0,1410,03-18-2022,,True,Unknown,Unknown,,Unknown,,Unknown,,False,,
1,1,1619,03-16-2022,JB,True,Swerve,5-More than 2 Cargo combination,Setup on the opposite side as the hangar in th...,Upper Hub only,They like the launchpad when heavily defended....,Traversal,15 sec climb. They go into the Hangar with 30 ...,False,Never seen them play defense.,
2,2,2945,03-20-2022,NG,True,Unknown,3-Shoot + Taxi only (1 Cargo),,Upper Hub only,,Mid,Usually go in to climb with 30sec left.,True,Not very effective,
3,3,3374,03-20-2022,NG,True,Unknown,4-Shoot + Taxi + Intake (2 Cargo max),,Upper Hub only,,Mid,,False,,Shoot close to the tarmac tape.\nCan easily ge...
4,4,4418,03-20-2022,NG,True,Unknown,4-Shoot + Taxi + Intake (2 Cargo max),"Shoot low, then taxi to pick up another ball, ...",Lower Hub only,Typically shoot from up against the hub.,None,"Trying to climb to the mid bar, but they get s...",False,,
5,5,4550,03-20-2022,NG,False,Unknown,3-Shoot + Taxi only (1 Cargo),Changed their auto routine to shoot into the l...,Nowhere,Did not pick up or shoot at all during teleop ...,Traversal,Enter hangar at 30sec. Have to enter the hanga...,True,Good at blocking. Never got penalties (except ...,In the first couple qual matches their robot s...
6,6,5690,03-20-2022,NG,True,Unknown,4-Shoot + Taxi + Intake (2 Cargo max),Looked like they were working on picking up a ...,Upper Hub only,,Mid,,True,Did well in finals when playing defense,It takes them awhile to line up to shoot. They...


In [8]:
# Let's see how many matches and what types of columns we have.
df_allpitstrategies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Index                     7 non-null      int64 
 1   Team                      7 non-null      int64 
 2   Timestamp                 7 non-null      object
 3   Scout                     7 non-null      object
 4   HARDWARE Floor Intake     7 non-null      bool  
 5   HARDWARE Drivetrain       7 non-null      object
 6   AUTO Level                7 non-null      object
 7   AUTO Strategy Comments    7 non-null      object
 8   TELEOP Where They Shoot   7 non-null      object
 9   TELEOP Strategy Comments  7 non-null      object
 10  END GAME Highest Climb    7 non-null      object
 11  END GAME Climb Comments   7 non-null      object
 12  DEFENSE Shown             7 non-null      bool  
 13  DEFENSE Comments          7 non-null      object
 14  Additional Notes          7 no

In [9]:
# Let's transpose our data to have all the Pit fields along the left side and the teams in a column
dfT_allpitstrategies = df_allpitstrategies.T

## Output the full list of Pit submissions we have from the Scouts

In [10]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
df_allpitstrategies[['Team','Timestamp','Scout','HARDWARE Floor Intake','HARDWARE Drivetrain','AUTO Level','AUTO Strategy Comments','TELEOP Where They Shoot','TELEOP Strategy Comments','END GAME Highest Climb','END GAME Climb Comments','DEFENSE Shown','DEFENSE Comments','Additional Notes']].to_csv("LatestResults/AllTeamStrategies.csv", index=False, float_format="%.1f")

In [11]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
dfT_allpitstrategies.to_csv("LatestResults/AllTeamStrategies_FieldsOnLeft.csv", index=False, float_format="%.1f")

In [12]:
# Attempting to write to Excel to look at AS-IS 
filename_Excel = "LatestResults/OurScouting-TeamStrategies.xlsx"
with pd.ExcelWriter(filename_Excel, datetime_format='mmm d yyyy hh:mm:ss', date_format='hh:mm:ss.000') as writer:
    dfT_allpitstrategies.to_excel(writer, sheet_name="Team Strategies-inColumns", index=True, float_format="%.1f")
    df_allpitstrategies[['Team','Timestamp','Scout','HARDWARE Floor Intake','HARDWARE Drivetrain','AUTO Level','AUTO Strategy Comments','TELEOP Where They Shoot','TELEOP Strategy Comments','END GAME Highest Climb','END GAME Climb Comments','DEFENSE Shown','DEFENSE Comments','Additional Notes']].to_excel(writer, sheet_name="Team Strategies-inRows", index=False, float_format="%.1f")
